In [1]:
import os
import openai
import sys

In [2]:
openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.document_loaders import ConfluenceLoader

In [ ]:
!pip install lark

In [4]:
loader = ConfluenceLoader(
    url="https://studytube.atlassian.net/wiki", username=os.environ['USERNAME'], api_key=os.environ['API_KEY']
)
documents = loader.load(space_key="ST", include_attachments=False, limit=50)

In [5]:
print(len(documents))

110


In [ ]:
#tech_documents = loader.load(space_key="TECH", include_attachments=False, limit=50)

In [ ]:
#print(len(tech_documents))

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 600,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

In [7]:
splitts = text_splitter.split_documents(documents)

In [8]:
len(splitts)

719

In [9]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [10]:
from langchain.vectorstores import Chroma

In [11]:
persist_directory='cache'

In [12]:
db = Chroma.from_documents(documents=splitts,
    embedding=embeddings_model,
    persist_directory=persist_directory)
db.persist()

In [13]:
print(db._collection.count())

4204


In [14]:
db.similarity_search("What is our password policy?")

[Document(page_content='Why do we use a password manager? In line with the Information Security Principles we use 1Password as our company-wide password management tool. Security is a vital part of our road to the BHAG. Product can build the best platforms, sales can work their hardest to close deals, but if we have a security breach, customers will lose trust in our platform and our work would be for nothing. 1Password', metadata={'id': '2193752086', 'source': 'https://studytube.atlassian.net/wiki/spaces/ST/pages/2193752086/1Password', 'title': '1Password'}),
 Document(page_content='Why do we use a password manager? In line with the Information Security Principles we use 1Password as our company-wide password management tool. Security is a vital part of our road to the BHAG. Product can build the best platforms, sales can work their hardest to close deals, but if we have a security breach, customers will lose trust in our platform and our work would be for nothing. 1Password', metadat

In [15]:
print(docs[2])

NameError: name 'docs' is not defined

In [25]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """
Imagine yourself as HR employee working in studytube. You need to answer the questions from another employee. 
Answer should based on information from confluence page. If possible add link to confluence page
Answer the question in form of steps needed to take.

{context}

Question: {question}
Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [17]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [18]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [19]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The URL to the cofluence page with detailed information`",
        type="string",
    ),
    AttributeInfo(
        name="title",
        description="The confluence page  title",
        type="string",
    ),
]

In [20]:
document_content_description = "Confluence pages"
retriever = SelfQueryRetriever.from_llm(
    OpenAI(temperature=0),
    db,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [21]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [27]:
retriever.get_relevant_documents(question)

[Document(page_content='party) to Chris or Victoria. Read also: Security incidents - when and how to report them . Only use complex and unique passwords, generated and stored by 1Password. Never share passwords via Slack or E-mail. Blur personal data in screenshots before uploading and sharing with colleagues . Limit the sending of personal data via e-mail as much as possible. Always share data internally via a Google Drive link. Ensure that your laptop requires a login after it goes to sleep or goes to screensaver. Lock your laptop whenever you leave it unattended. Never leave portable hardware like laptops or', metadata={'id': '2189492650', 'source': 'https://studytube.atlassian.net/wiki/spaces/ST/pages/2189492650/Information+Security+Principles+and+your+Duty+of+Confidentiality', 'title': 'Information Security Principles and your Duty of Confidentiality'}),
 Document(page_content='party) to Chris or Victoria. Read also: Security incidents - when and how to report them . Only use comp

In [22]:
question='Which is our password policy?'

In [28]:
result = qa_chain({"query": question})

In [29]:
print(result["result"])

Our password policy is as follows:

1. Only use complex and unique passwords.
2. Generate and store passwords using 1Password.
3. Never share passwords via Slack or email.
4. Do not use the same password in multiple places.
5. Blur personal data in screenshots before uploading and sharing with colleagues.
6. Limit the sending of personal data via email as much as possible.
7. Always share data internally via a Google Drive link.
8. Ensure that your laptop requires a login after it goes to sleep or goes to screensaver.
9. Lock your laptop whenever you leave it unattended.
10. Never leave portable hardware like laptops or passwords unattended.
